<a href="https://colab.research.google.com/github/ga-pontes/tcc/blob/main/GDELT_Agrupamento_com_restri%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Printar tempo de execução


In [ ]:
!pip install ipython-autotime
%load_ext autotime

# Autenticação Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from datetime import date
today = date.today().strftime("%Y%m%d")

In [ ]:
today

In [ ]:
cd gdrive/My\ Drive/tcc

In [ ]:
import os
os.mkdir(today)

In [ ]:
cd ..

# Carregar tabelas e vetores 
*Seção auxiliar para rotina de testes*


## Exemplos de funções

### Salvar

#### Salvar CSV Pandas


In [ ]:
file = '/content/gdrive/My Drive/tcc/'+today+'/tabela.csv'
df.to_csv(file)

#### Salvar serial Pickle

In [ ]:
import pickle

file = '/content/gdrive/My Drive/tcc/'+today+'/embeddings_array_7827'

with open(file, 'wb') as f:
  pickle.dump(embeddings_array, f)

### Carregar

#### Carregar CSV Pandas

In [ ]:
#exemplo de leitura
import pandas as pd 
eventos = pd.read_csv('/content/gdrive/My Drive/tcc/20201013/uscities_5647.csv') 

#### Carregar serial pickle

In [ ]:
import pickle

file = '/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/restricoes_1/embeddings_distances'

with open (file, 'rb') as fp:
    embeddings_distances = pickle.load(fp)

## Tabelas de eventos

In [ ]:
#exemplo de leitura
import pandas as pd 
eventos = pd.read_csv('/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/restricoes_1/uscities_5647.csv') 

In [ ]:
eventos.head(3)

In [ ]:
print("Eventos: " + str(eventos.shape[0]))
print("Temas: " + str(eventos.theme.nunique()))

## Embedding Array


In [ ]:
import pickle

file = '/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/restricoes_1/embeddings_array'

with open (file, 'rb') as fp:
    embeddings_array = pickle.load(fp)

In [ ]:
len(embeddings_array)

# Autenticação BigQuery

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Carregamento da tabela de eventos

In [ ]:
%load_ext google.colab.data_table

In [ ]:
%%bigquery --project tcc-gdelt eventos
SELECT
  Title,
  array_agg(locations_mentioned[safe_offset(0)].loc_fullname order by Partitiondate desc LIMIT 1)[safe_offset(0)] loc_fullname,
  array_agg(locations_mentioned[safe_offset(0)].loc_latitude order by Partitiondate desc LIMIT 1)[safe_offset(0)] loc_latitude,
  array_agg(locations_mentioned[safe_offset(0)].loc_longitude order by Partitiondate desc LIMIT 1)[safe_offset(0)] loc_longitude,
  array_agg(ThemesV2_Array[safe_offset(0)].theme order by Partitiondate desc LIMIT 1)[safe_offset(0)] theme
FROM
  `tcc-gdelt.dataset_projeto.gkg_join_gdelt_uscities`
where 
  partitiondate between '2020-07-01' and '2020-07-30'
  and (MOD(CAST(RAND()*10 AS INT64),3)=0) 
  and ThemesV2_Array[safe_offset(0)].theme in (SELECT DISTINCT tema FROM `tcc-gdelt.themes_hierarchy.poda_temas`)
  and Title <> ''
  and regexp_extract(locations_mentioned[safe_offset(0)].loc_adm1code, 'US(.*)') NOT in ('AK', 'HI', 'PR')
GROUP BY
  Title

In [ ]:
print(eventos['Title'].nunique())
print(eventos['theme'].nunique())

In [ ]:
eventos.head()

In [ ]:
print(eventos.shape[0])

In [ ]:
eventos.to_csv('/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/uscities_3936.csv')

#Funções para geração de restrições





### **Must link:** eventos com mesmo local





In [ ]:
import itertools 
def generate_constrains_same_local_name(eventos, campo_local):
    lista_locais = eventos[campo_local].values.tolist()

    locations_dict = {}
    for index, local_name in enumerate(lista_locais):
        if local_name not in locations_dict:
            locations_dict[local_name] = {index}
        else:
            locations_dict[local_name].add(index)

    #print(locations_dict)

    must_link = []
    for loc in locations_dict: 
      #print(local_name)
      b = list(itertools.combinations(list(locations_dict[loc]), 2))
      must_link.extend(b)
      #print(b)

    return must_link

In [ ]:
eventos.head(0)

In [ ]:
must_link = generate_constrains_same_local_name(eventos, 'loc_fullname')

In [ ]:
print("## de Restrições ML, sem considerar conteúdo textual: " + str(len(must_link)))

### **Refinar Must link:** eventos com mesmo local e previamente agrupados

In [ ]:
def must_link_filter(eventos, ml, clusters_column):
  lista_clusters = eventos[clusters_column].values.tolist()
  ml_filter = []
  for comb in ml:
    if lista_clusters[comb[0]] == lista_clusters[comb[1]]:
      ml_filter.append(comb)

  return ml_filter

In [ ]:
eventos.head(0)

In [ ]:
mlf = must_link_filter(eventos, must_link,  'clusters35')

In [ ]:
print(len(must_link))
print(len(mlf))

### **Cannot link**: eventos mais distantes que limiar de distância



In [ ]:
!pip install haversine

In [ ]:
from haversine import haversine, Unit

def removeDuplicates(lst):  
    return [t for t in (set(tuple(i) for i in lst))]

def generate_cl_hav(events, k, taxa_de_poda, campo_lat, campo_lon):
  # EVENTS: tabela de eventos
  # K: subdivisões máximas do espaço geográfico
  # taxa_de_poda: restrição às subdivisões do espaços geográfico 
  # varia de 0 a 1: 
  #     - 0 não aplica restrição de distância
  #     - 1 só permite agrupamento com eventos dentro do raio de subdivisão
  #     - 0,5 avança 50% da distância entre o raio mínimo de subdivisão e o espaço total

  #Definição do beta

  
  lat_list = [float(i) for i in eventos['loc_latitude']]
  lon_list = [float(i) for i in eventos['loc_longitude']]

  Lat_min = min(lat_list)
  Lat_max = max(lat_list)
  Lon_min = min(lon_list)
  Lon_max = max(lon_list)

  zero = (Lat_min, Lon_min)
  print(zero)
  pico = (Lat_max, Lon_max)
  print(pico)

  diagonal = haversine(zero, pico)
  print("Diagonal do espaço amostral: " + str(diagonal))
  beta = (diagonal)*(1 + (taxa_de_poda)*(1/k - 1))
  print("Beta definido: " + str(beta))

  #Criação da lista de (lat, lon)
  geo_list = []
  geo_list.extend((lat,lon) for lat, lon in zip(lat_list, lon_list))

  #Criação da lista de listas com a distância entre cada local da lista de eventos
  geo_list_havs = []
  for geo in geo_list:
    geo_list_havs.append([haversine(geo,x) for x in geo_list])

  print("ok")

  #Verificação dos eventos mais distantes do que o permitido
  cl=[]
  for index1, elem1 in enumerate(geo_list_havs):
    for index2, elem2 in enumerate(elem1):
      if index1 != index2:
        if geo_list_havs[index1][index2] > beta:
          if index1 <= index2:
            link = (int(index1), int(index2))
          else:
            link = (int(index2), int(index1))
          cl.append(link)

  cannot_link_array_dedouble = removeDuplicates(cl)
  
  return cannot_link_array_dedouble

In [ ]:
# restring que um evento possar estar no maximo 75% do diametro do espaço distante de outro evento
# 1 não convergiu
clf = generate_cl_hav(eventos, 4, 0.50, 'loc_latitude', 'loc_longitude') 

In [ ]:
print("Restrições de agrupamentos geradas: " + str(len(clf)))

### **Afrouxar Cannot link**: eventos de um mesmo clusters que foram restritos mas que tem transitividade


In [ ]:
def generate_cluster_set(lista_clusters):
  clusters_set = {}
  for index, cluster in enumerate(lista_clusters):
    if cluster not in clusters_set:
      clusters_set[cluster] = [index]
    else:
      clusters_set[cluster].append(index)

  return clusters_set

def generate_cannot_link_set(cannot_link):
  cannot_link_set = {}
  
  for index, tupla_cl in enumerate(cannot_link):
    print("Iteração: " + str(index))
    if tupla_cl[0] not in cannot_link_set:
      cannot_link_set[tupla_cl[0]] = [tupla_cl[1]]
    else:
      if tupla_cl[1] not in cannot_link_set[tupla_cl[0]]:
        cannot_link_set[tupla_cl[0]].append(tupla_cl[1])

    if tupla_cl[1] not in cannot_link_set:
      cannot_link_set[tupla_cl[1]] = [tupla_cl[0]]
    else:
      if tupla_cl[0] not in cannot_link_set[tupla_cl[1]]:
        cannot_link_set[tupla_cl[1]].append(tupla_cl[0])

  return cannot_link_set

def soften_cannot_link(tabela, campo_clusters, cannot_link):
  clusters_set = generate_cluster_set(tabela[campo_clusters])
  cannot_link_set = generate_cannot_link_set(cannot_link)
  out_cannot_link_softer = cannot_link.copy()

  for cannot_link_tuple in cannot_link:
    evento_a = cannot_link_tuple[0]
    evento_b = cannot_link_tuple[1]
    cluster_evento_a = tabela[campo_clusters].loc[evento_a]
    cluster_evento_b = tabela[campo_clusters].loc[evento_b]
  
    if cluster_evento_a == cluster_evento_b:
      for cluster_event in clusters_set[cluster_evento_a]:
        if (cluster_event != evento_a) and (cluster_event != evento_b):
          print('check cluster_event: ' + str(cluster_event))
          if (cluster_event in cannot_link_set) and (evento_a not in cannot_link_set[cluster_event]) and (evento_b not in cannot_link_set[cluster_event]):
            out_cannot_link_softer.remove((evento_a, evento_b))
            break
  return out_cannot_link_softer

In [ ]:
soft = soften_cannot_link(eventos, 'clusters', cl)

In [ ]:
print(len(cl))
print(len(soft))

### **MÉTODO FINAL** - Distancia semântica e geográfica

In [ ]:
#distância euclidiana do kmeans
def l2_distance(point1, point2):
    return sum([(float(i)-float(j))**2 for (i, j) in zip(point1, point2)])

In [ ]:
#gerar todas as combinações 2 a 2 de eventos
from itertools import combinations
def comibinatoria_de_2(e_array):
  indices = [i for i in range(len(e_array))]
  comb = list(combinations(indices, 2))
  return comb

In [ ]:
#calcular as distancias entre os eventos a partir da saida do bert
def bert_distances(e_array, comb):
  embd = []
  for index, c in enumerate(comb):
    embd.append(l2_distance(embeddings_array[c[0]],embeddings_array[c[1]]))
    if index % 10000 == 0:
      print("Iteração: " + str(index)) 
  return embd, min(embd), max(embd)

In [ ]:
#normalizar as distancias entre os eventos a partir da saida do bert
def normalize_bert_distances(bert_dist, b_min, b_max):
  normal_bert = []
  for dist in bert_dist:
    normal_bert.append( (dist - b_min) / (b_max - b_min))
  return normal_bert

In [ ]:
combinacoes = comibinatoria_de_2(embeddings_array)
len(combinacoes)

In [ ]:
len(embeddings_array)

In [ ]:
b_embeddings_distances, dist_min, dist_max = bert_distances(embeddings_array, combinacoes)

In [ ]:
b_embeddings_distances = embeddings_distances
dist_min = min(b_embeddings_distances)
dist_max = max(b_embeddings_distances)

In [ ]:
len(embeddings_distances)

In [ ]:
import pickle
file = '/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/embeddings_distances'

with open(file, 'wb') as f:
  pickle.dump(b_embeddings_distances, f)

In [ ]:
normal_berts = normalize_bert_distances(b_embeddings_distances, dist_min, dist_max)
len(normal_berts)

In [ ]:
!pip install haversine

In [ ]:
from haversine import haversine, Unit

def haversine_distances(events, comb):
  lat_list = [float(i) for i in eventos['loc_latitude']]
  lon_list = [float(i) for i in eventos['loc_longitude']]
  geo_list = []
  geo_list.extend((lat,lon) for lat, lon in zip(lat_list, lon_list))

  haver_dists = []
  for index, c in enumerate(comb):
    if index % 1000000 == 0:
      print("Iteração: " + str(index)) 
    haver_dists.append( haversine(geo_list[c[0]], geo_list[c[1]]))

  mx = max(haver_dists)
  haver_dists_normalized = [x/mx for x in haver_dists]

  return haver_dists_normalized

In [ ]:
distancias_hav = haversine_distances(eventos, combinacoes)

In [ ]:
print(len(distancias_hav))
print(len(b_embeddings_distances))
print(len(combinacoes))

In [ ]:
zip_combinations = list(zip(combinacoes, distancias_hav, normal_berts))

In [ ]:
zip_combinations_sort = zip_combinations.copy()
zip_combinations_sort.sort(key=lambda tp: tp[2])

In [ ]:
combs_sort = []
distancias_hav_sort = []
normal_berts_sort = []

for t in zip_combinations_sort:
  combs_sort.append(t[0])
  distancias_hav_sort.append(t[1])
  normal_berts_sort.append(t[2])

In [ ]:
array_aux = []
for index, tpl in enumerate(distancias_hav_sort):
  if tpl >= 0.75:
    array_aux.append("CL")
  #elif normal_berts_sort[index] <= 0.02 and tpl == 0:
   # array_aux.append("ML")
  else:
    array_aux.append("Sem restrição")

In [ ]:
len(array_aux)

In [ ]:
dits_hav_sort = distancias_hav_sort.copy()
dits_hav_sort.sort()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot([i for i in range(len(normal_berts_sort))],normal_berts_sort)
plt.title('Distribuição de distância semântica entre par de eventos (RSM)')
plt.xlabel('Índice da dupla na lista')
plt.ylabel('Distância (euclidiana) da dupla')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(num=None, figsize=(12, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot([i for i in range(len(dits_hav_sort))],dits_hav_sort)
plt.title('Distribuição de distância geográfica entre par de eventos (RGEO)')
plt.xlabel('Índice da dupla na lista')
plt.ylabel('Distância (haversine) da dupla')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="ticks")

x = distancias_hav_sort[0:96700]
y = normal_berts_sort[0:96700]

p = sns.jointplot(x=x, y=y, hue=array_aux[0:96700])
p.fig.set_size_inches(10,10)

p.set_axis_labels('x', 'y', fontsize=16)
p.ax_joint.set_xlabel("Distribuição de distâncias geográficas", fontweight='bold')
p.ax_joint.set_ylabel("Distribuição de distâncias semânticas", fontweight='bold')

In [ ]:
i_max = 0
while normal_berts_sort[i_max] < 0.4:
  i_max = i_max + 100

print(i_max)

In [ ]:
ml = []
cl = []

for index, c in enumerate(combs_sort[0:50500]):
  if distancias_hav_sort[index] >= 0.7:
    cl.append(c)
  if distancias_hav_sort[index] <= 0.001:
    ml.append(c)

print(len(ml))
print(len(cl))

In [ ]:
ml = []

for index, c in enumerate(combs_sort[0:34900]):
  if distancias_hav_sort[index] <= 0.02 and c in must_link:
    ml.append(c)

print(len(ml))

In [ ]:
print(len(ml))

In [ ]:
matchs_ml = 0
for i in ml:
  if eventos['loc_fullname'].loc[i[0]] == eventos['loc_fullname'].loc[i[1]]:
    matchs_ml = matchs_ml + 1
print(matchs_ml)

In [ ]:
print(matchs_ml/len(ml))

In [ ]:
matchs_cl = 0
for i in cl:
  if eventos['theme'].loc[i[0]] != eventos['theme'].loc[i[1]]:
    matchs_cl = matchs_cl + 1
print(matchs_cl)

In [ ]:
print(matchs_cl/len(cl))

#Carregamento do Sentence Transformer


### Download e instalação de pacotes

In [ ]:
ls

In [ ]:
# instalando e carregando sentencen transformer

!git clone https://github.com/UKPLab/sentence-transformers.git
!pip install -U sentence-transformers/

In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging

#### Just some code to print debug information to stdout
np.set_printoptions(threshold=100)

logging.basicConfig(format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO, handlers=[LoggingHandler()])

# Load Sentence model (based on BERT) from URL
model = SentenceTransformer('distiluse-base-multilingual-cased')

### Transformando eventos em vetores

In [ ]:
eventos.head(0)

In [ ]:
event_title_array = eventos['Title'].values.tolist()
event_title_array

In [ ]:
embeddings_array = model.encode(event_title_array)
embeddings_array

In [ ]:
for i, eb in enumerate(embeddings_array):
  if eb.any() != embeddings_array2[i].any():
    print(ops)

####Salvar embedding_array

In [ ]:
import pickle

file = '/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/embeddings_array'

with open(file, 'wb') as f:
  pickle.dump(embeddings_array, f)

####Carregar embedding_array



**Código na seção:** Carregar tabelas e vetores > Embedding Array

#Carregamento do COP-KMeans
Download e instalação de pacotes

In [ ]:
ls

In [ ]:
!git clone https://github.com/Behrouz-Babaki/COP-Kmeans.git
!pip install COP-Kmeans/

#Agrupamento sem restrições

In [ ]:
print('Quantidade de eventos testados: ' + str(len(embeddings_array)))

In [ ]:
#Verificação de parâmetros
if embeddings_array != []:
  print("Embeddings ok.")
else:
  print("Array de embeddings não carregado.")

In [ ]:
## agrupamento sem restricoes
from copkmeans.cop_kmeans import cop_kmeans
import numpy as np

k = 50
input_matrix = np.array(embeddings_array)
clusters, centers = cop_kmeans(dataset=input_matrix, k=k, ml=[], cl=[])


if clusters != None:
  print("Sucesso!")
else: 
  print("Não convergiu.")

In [ ]:
import pickle

file = '/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/clusters50'
with open(file, 'wb') as f:
  pickle.dump(clusters, f)

In [ ]:
clusters

In [ ]:
import pandas as pd

resultados = pd.DataFrame(data=eventos)
resultados['clusters35'] = clusters
resultados.to_csv('/content/gdrive/My Drive/tcc/'+today+'/35_resultado_sem_restricoes.csv')

In [ ]:
eventos.head(3)

#Agrupamento com restrições


In [ ]:
#Verificação de parâmetros
if embeddings_array != []:
  print("Embeddings ok.")
else:
  print("Array de embeddings não carregado.")

if ml != []:
  print("ML ok.")
else:
  print("Array ML não carregado.")

if cl != []:
  print("CL ok.")
else:
  print("Array CL não carregado.")

In [ ]:
#Verificação de sobreposição entre ML e CL
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 
    
if intersection(cl, ml) == []:
  print("Sem intersecção.")
else:
  print("Erro! Há interseccção.")

In [ ]:
clusters = []
centers = []
clusters.clear() 
centers.clear()
print(clusters)
print(centers)

In [ ]:
embeddings_array

In [ ]:
from copkmeans.cop_kmeans import cop_kmeans
import numpy as np

k = 50
input_matrix = np.array(embeddings_array)
#clusters, centers = cop_kmeans(dataset=input_matrix, k=k, ml=ml[0:4000], cl=cl)
clusters, centers = cop_kmeans(dataset=input_matrix, k=k, ml=[], cl=cl)

if clusters != None:
  print("Sucesso!")
else: 
  print("Não convergiu.")

In [ ]:
len(ml[0:3000])

In [ ]:
import pickle

file = '/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/clusters30r'
with open(file, 'wb') as f:
  pickle.dump(clusters, f)

# Avalição dos Clusters

In [ ]:
import pickle

file = '/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/clusters50'

with open (file, 'rb') as fp:
    clusters50 = pickle.load(fp)

In [ ]:
print(eventos.shape[0])
print(len(clusters40))
print(len(clusters40r))

In [ ]:
eventos.head(0)

In [ ]:
from sklearn import metrics
from sklearn.metrics.cluster import adjusted_rand_score
print(metrics.normalized_mutual_info_score(eventos['theme'].to_list(), clusters50))
print(metrics.normalized_mutual_info_score(eventos['loc_fullname'].to_list(), clusters50))

In [ ]:
clusters30rml_b

In [ ]:
from sklearn import metrics
from sklearn.metrics.cluster import adjusted_rand_score
print(metrics.normalized_mutual_info_score(eventos['theme'].to_list(), clusters60rml_b))

print(metrics.normalized_mutual_info_score(eventos['loc_fullname'].to_list(), clusters60rml_b))


In [ ]:
from sklearn import metrics
from sklearn.metrics.cluster import adjusted_rand_score
print(metrics.normalized_mutual_info_score(eventos['theme'].to_list(), clusters50rml_b))

print(metrics.normalized_mutual_info_score(eventos['loc_fullname'].to_list(), clusters50rml_b))


In [ ]:
##### recomendado estudar e usar esses baseados em mutual_information (estudar o range da medida? quanto maior melhor?)
from sklearn.metrics.cluster import mutual_info_score

print(metrics.mutual_info_score(eventos['loc_fullname'].to_list(), clusters40r))

In [ ]:
from sklearn.metrics.cluster import mutual_info_score

print(metrics.adjusted_mutual_info_score(eventos['theme'].to_list(), clusters35))

### Adicionar ao DF

In [ ]:
import pandas as pd

eventos_novo = pd.DataFrame(data=eventos)
eventos_novo['nova_coluna'] = clusters
resultados_fortes3.to_csv('/content/gdrive/My Drive/tcc/20200927/resultado_com_restricoes_fortes4.csv')

### Salvar novo DF

**Instruções na seção:** Carregar tabelas e vetores > Carregar > Salvar CSV Pickle

## Mapas de Calor

In [ ]:
import plotly.express as px
#fig = px.scatter_geo(data_frame=df, 
#                     locations="iso_alpha", 
#                     color="continent",
#                     hover_name="country", size="pop",
#                     animation_frame="year",
#                     projection="natural earth")
#

fig = px.scatter_geo(data_frame=eventos, 
            lat=eventos['loc_latitude'], 
            lon=eventos['loc_longitude'], 
            locations=None, 
            locationmode=None, 
            color=ev2, 
            text=None, 
            hover_name=None, 
            hover_data=None, 
            custom_data=None, 
            size=None, 
            animation_frame=None, 
            animation_group=None, 
            category_orders={}, 
            labels={}, 
            color_discrete_sequence=None, 
            color_discrete_map={}, 
            color_continuous_scale=None, 
            range_color=None, 
            color_continuous_midpoint=None, 
            opacity=None, 
            size_max=None, 
            projection=None, 
            scope='usa', 
            center=None, 
            title=None, 
            template=None, 
            width=2000, 
            height=1000)
fig.show()

In [ ]:
import pickle

file = '/content/gdrive/My Drive/tcc/testes_modelo_novo/tabela1/restricoes_2/clusters50rml_b'

with open (file, 'rb') as fp:
    clusters50rml_b = pickle.load(fp)

In [ ]:
eventos['clusters_r'] = clusters50rml_b

In [ ]:
eventos.head(3)

In [ ]:
eventos['loc_latitude'].loc[eventos['clusters'] == 13]

In [ ]:
int(len(lats)*0.75)

In [ ]:
import folium

# plotando eventos no mapa
m = folium.Map([45.5236, -122.6750], zoom_start=4)

tipo_de_agrupamento = 'clusters'

cluster = 3
lats = eventos['loc_latitude'].loc[eventos[tipo_de_agrupamento] == cluster].tolist()
lngs = eventos['loc_longitude'].loc[eventos[tipo_de_agrupamento] == cluster].tolist()

for i in range(int(len(lats)*0.75)):
  lat = lats[i]
  lng = lngs[i]
  folium.Marker([lat, lng], icon=folium.Icon(icon='circle', color='purple')).add_to(m)

cluster = 4
lats = eventos['loc_latitude'].loc[eventos[tipo_de_agrupamento] == cluster].tolist()
lngs = eventos['loc_longitude'].loc[eventos[tipo_de_agrupamento] == cluster].tolist()

for i in range(int(len(lats)*0.75)):
  lat = lats[i]
  lng = lngs[i]
  folium.Marker([lat, lng], icon=folium.Icon(icon='circle', color='blue')).add_to(m)

cluster = 34
lats = eventos['loc_latitude'].loc[eventos[tipo_de_agrupamento] == cluster].tolist()
lngs = eventos['loc_longitude'].loc[eventos[tipo_de_agrupamento] == cluster].tolist()

for i in range(int(len(lats)*0.75)):
  lat = lats[i]
  lng = lngs[i]
  folium.Marker([lat, lng], icon=folium.Icon(icon='circle', color='green')).add_to(m)

cluster = 42
lats = eventos['loc_latitude'].loc[eventos[tipo_de_agrupamento] == cluster].tolist()
lngs = eventos['loc_longitude'].loc[eventos[tipo_de_agrupamento] == cluster].tolist()

for i in range(int(len(lats)*0.6)):
  lat = lats[i]
  lng = lngs[i]
  folium.Marker([lat, lng], icon=folium.Icon(icon='circle', color='orange')).add_to(m)

m

In [ ]:
import folium
from folium.plugins import HeatMap

cluster = 42
lats = eventos['loc_latitude'].loc[eventos['clusters'] == cluster].tolist()
lngs = eventos['loc_longitude'].loc[eventos['clusters'] == cluster].tolist()

m = folium.Map([40.0995, -99.1351], zoom_start=4)
data_heatmap = []

for i in range(len(lats)):
  lat = lats[i]
  lng = lngs[i]
  data_heatmap.append([lat,lng,1])

HeatMap(data_heatmap).add_to(m)
m

In [ ]:
lats = eventos['loc_latitude'].loc[eventos['clusters'] == 13].tolist()
longs = eventos['loc_longitude'].loc[eventos['clusters'] == 13].tolist()

for i in range(len(lats)):
  print(lats[i])
  print(longs[i])